# 03. 크롤링한 text data를 전처리해서 저장하기
- 작성 : 정민정 (https://github.com/jeina7)

---
- 지금까지 수집한 모든 plain text 데이터를 일차적으로 전처리한다.


- 특정 필요한 특수문자 (`"."`, `","` 등의 구두점 등) 를 제외한 다른 특수문자는 모두 제거


- 자모음 (`'ㅋㅋㅋ'`, `'ㅜㅜ'` 등) 또한 우리가 생성할 수필 데이터에 부합하지 않으므로 모두 제거
---

### 1. 현재 디렉토리 상황 확인

In [1]:
!tree -d

.
├── Crawling
│   └── textcrawler
│       ├── spiders
│       └── text_data
├── common
│   ├── data
│   ├── models
│   │   └── 345K
│   └── src
│       └── __pycache__
└── no_use_notebooks

11 directories


- `common` 은 사전에 학교측에서 제공받은 코드 모음


- 우리가 수집한 crawling 코드는 `Crawling` 디렉토리에 있음


- 이 중 현재까지 수집한 text 데이터는 `Crawling/textcrawler/text_data` 내에 위치

In [2]:
!ls Crawling/textcrawler/text_data/

data.txt       lifeCrawl.csv  sinchun.txt    storyCrawl.csv


- `lifeCrawl.csv`, `storyCrawl.csv`, `sinchun.csv` 세 가지 파일로 존재 (수필 / 소설)

### 2. text를 전처리하는 함수 `PreProcess`
- 필요없는 부분 제거
- 자모음 제거
- `.`, `,`, `'`, `"`, `?`, `!` 를 제외한 특수문자 제거

In [3]:
import re

def PreProcess(text):
    text = re.sub(pattern='Posted on [0-9]{4} [0-9]{2} [0-9]{2} .+ Posted in \S+ \s?', repl='', string=text)
    _filter = re.compile('[ㄱ-ㅣ]+')
    text = _filter.sub('', text)
    _filter = re.compile('[^가-힣 0-9 a-z A-Z \. \, \' \" \? \!]+')
    text = _filter.sub('', text)
    return text

In [4]:
a = ".. 아아 ? >>"
PreProcess(a)

'.. 아아 ? '

### 3. 데이터를 합쳐서 저장하는 함수 `save_data`

In [5]:
from glob import glob

def save_data(dir_path, save_path):
    files = [f for f in glob(dir_path + "*", recursive=True)]
    
    data = ''
    
    for file in files:
        suffix = file.split("/")[-1].split(".")[-1]
        
        if suffix == 'csv':
            df = pd.DataFrame.from_csv(file).reset_index()
            print('{} data saving. size:'.format(file.split('/')[-1]), df.shape[0])

            for i, text in enumerate(df['content'].values):
                text = PreProcess(text)
                df.loc[i, 'content'] = text

            data += "\n".join(df['content'].values)
            
        elif (suffix == 'txt') and (not file.split("/")[-1].startswith("data")):
            print('{} data saving.'.format(file.split('/')[-1]))
            with open(file, 'r') as f:
                data += f.read()
                
    with open(save_path, 'w') as f:
        f.write(data)

    print("\nAll saved.".format(dir_path.split('/')[-1]))

### 4. 합칠 데이터와 새로 저장할 데이터 Path 지정

In [6]:
dir_path = './Crawling/textcrawler/text_data/'
save_path = './Crawling/textcrawler/text_data/data.txt'

### 5. 함수 실행

In [7]:
save_data(dir_path, save_path)

sinchun.txt data saving.
lifeCrawl.csv data saving. size: 2435
storyCrawl.csv data saving. size: 3957

All saved.


### 6. 저장된 데이터 확인

In [8]:
with open(save_path, 'r') as f:
    data = f.read()

data[:500]

'“야, 이 썩을 놈아! 시끄럽다고 몇 번이나 말하노? 어이?”\n현관문을 부서져라 두드리며 욕할매가 고함을 친다. 나는 그러거나 말거나 문도 열어보지 않는다. 저러다가 시들해져 집에 간 적이 많아 이제 신경도 안 쓴다. 욕할매는 우리 아래층, 401호에 산다. 80살도 넘었다는데 목소리가 어찌나 우렁찬지 밖에서 소리를 지르면 우리 집이 쿵쿵 울리는 것 같다. 우리가 이사 오던 날부터 이삿짐 나르는 사다리차가 시끄럽다고 난리였다.\n“허구헌 날 다 나뚜고 우짠 일로 이 늦은 시간에 이사를 해 쌌소?”\n“죄송해요. 저희 부부가 다 일을 다니다 보니 이 시간에 할 수밖에 없네요.”\n우리가 이사를 하는 시간이 저녁때인 것은 맞다. 하지만 깜깜한 한밤중도 아닌데 뭐 그리 난리 법석을 할 일은 아니다. 욕할매는 이삿짐 나르는데 불쑥 들어와서 한바탕 하고 갔다. 그 일이 있은 후로 우리 집 가훈은 ‘살금살금, 조용조용’이 된 것 같았다.\n“아래층 할머니 너무 하신 거 아녜요?”\n엄마는 부엌을 정리하'